# Постановка задачи

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

# Преобразование данных в формат csv

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:
id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...
Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:
id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...
С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [1]:
import pandas as pd

In [22]:
data = []
lines = []

with open('checkins.dat', 'r') as input_file:
    columns_names = map(lambda x: x.strip(), input_file.readline().strip().split('|'))
    n_params = len(columns_names)
    for line in input_file:
        items = map(lambda x: x.strip(), line.strip().split('|'))
        if len(items) != n_params:
            print line
        else:
            data.append(items)      

---------+---------+----------+-------------------+-------------------+---------------------

(1021966 rows)





In [23]:
df = pd.DataFrame(data)
df.columns = columns_names
df = df[(df.latitude != '') & (df.longitude != '')]
df.shape

(396634, 6)

In [24]:
df.to_csv('data.csv')

In [25]:
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
7,984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22
9,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870,380645,33.4483771,-112.0740373,2012-04-21 17:38:18


# Кластеризация

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [35]:
from sklearn.cluster import MeanShift

data = df[['latitude', 'longitude']]
data = data.applymap(float)

In [36]:
data_sample = data.sample(100000)

In [37]:
%%time
clstr = MeanShift(bandwidth=0.1)
clstr.fit(data_sample)

Wall time: 3min 47s


Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:
38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...

In [39]:
clstr.predict(data_sample)

array([   1,  676,    3, ...,    4, 2699,   10], dtype=int64)

In [40]:
data_sample['cluster'] = clstr.predict(data_sample)

In [45]:
cluster_size = pd.DataFrame(data_sample.pivot_table(index = 'cluster', aggfunc = 'count', values = 'latitude'))
cluster_size.columns = ['cluster_size']

In [46]:
df_cluster_centers_ = pd.DataFrame(clstr.cluster_centers_)
df_cluster_centers_.columns = ['center_latitude', 'center_longitude']

In [47]:
clusters_df = df_cluster_centers_.join(cluster_size)
clusters_df.to_csv('clusters_df')
clusters_df  = clusters_df[clusters_df.cluster_size > 15]

In [49]:
clusters_df.head()

,center_latitude,center_longitude,cluster_size
0,40.717852,-73.989326,14135
1,33.449343,-112.002009,5087
2,41.878154,-87.629833,3821
3,38.886058,-77.048288,2806
4,37.685843,-122.409212,1634


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:
33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)
Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [50]:
def get_distance(latitude1, longitude1, latitude2, longitude2):
    return ((latitude1 - latitude2)**2 + (longitude1 - longitude2)**2)**0.5

office_coordinates = [
                    (33.751277, -118.188740),
                    (25.867736, -80.324116),  
                    (51.503016, -0.075479),  
                    (52.378894, 4.885084),  
                    (39.366487, 117.036146), 
                    (-33.868457, 151.205134)
] 
def get_min_distance(latitude, longitude):
    min_distance = None
    for (office_lat, office_lon) in office_coordinates:
        dist = get_distance(latitude, longitude, office_lat, office_lon)
        if (min_distance is None) or (dist < min_distance):
            min_distance = dist
    return min_distance

In [51]:
clusters_df['min_distance'] = map(get_min_distance, clusters_df.center_latitude, clusters_df.center_longitude)

In [60]:
clusters_df_20_values = clusters_df.sort_values('min_distance')[:20]
clusters_df_20_values

,center_latitude,center_longitude,cluster_size,min_distance
243,-33.866315,151.204861,54,0.002159
365,52.372878,4.892406,32,0.009477
52,51.502971,-0.126901,301,0.051422
187,25.911580,-80.269340,83,0.070162
48,33.809954,-118.146755,291,0.072151
24,25.787188,-80.215169,673,0.135490
92,26.005507,-80.209016,119,0.179524
84,33.902232,-118.064199,152,0.195698
36,33.873308,-118.369193,419,0.217841
3463,25.633994,-80.432277,17,0.257554


In [75]:
lat = clusters_df_20_values['center_latitude'].values[0]
lon = clusters_df_20_values['center_longitude'].values[0]

In [76]:
def write_answer(lat, lon):
    with open('banners_answer.txt', 'w') as f_out:
        f_out.write(str(lat)+' '+str(lon))

In [77]:
write_answer(lat, lon)